# **1. Instalando o Pyspark**


In [27]:
!pip install pyspark

# **2. Inicializando Pyspark**

In [2]:
import pyspark.sql as sql
from pyspark.sql.functions import *

In [3]:
spark = sql.SparkSession.builder.appName("Fraude_Ecommerce").getOrCreate()

# **3. Lendo o Dataset**

In [4]:
df = spark.read.csv("Fraud_Data.csv", header = True, inferSchema=True)

# Definindo um schema e criando o dataframe explicitamente.
#schema = StructType([
#    StructField("user_id", IntegerType(), nullable=True),
#    StructField("signup_time", TimestampType(), nullable=True),
#    StructField("purchase_time", TimestampType(), nullable=True),
#    StructField("purchase_value", IntegerType(), nullable=True),
#    StructField("device_id", StringType(), nullable=True),
#    StructField("source", StringType(), nullable=True),
#    StructField("browser", StringType(), nullable=True),
#    StructField("sex", StringType(), nullable=True),
#    StructField("age", IntegerType(), nullable=True),
#    StructField("ip_address", DoubleType(), nullable=True),
#    StructField("class", IntegerType(), nullable=True)
#])

#data = spark.read.csv("Fraud_Data.csv", header = True)
#df = spark.createDataFrame(data, schema)


In [5]:
df.show()

+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+-----+
|user_id|        signup_time|      purchase_time|purchase_value|    device_id|source|browser|sex|age|        ip_address|class|
+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+-----+
|  22058|2015-02-24 22:55:49|2015-04-18 02:47:11|            34|QVPSPJUOCKZAR|   SEO| Chrome|  M| 39| 7.3275836879972E8|    0|
| 333320|2015-06-07 20:39:50|2015-06-08 01:38:54|            16|EOGFQPIZPYXFZ|   Ads| Chrome|  F| 53|3.50311387865908E8|    0|
|   1359|2015-01-01 18:52:44|2015-01-01 18:52:45|            15|YSSKYOSJHPPLJ|   SEO|  Opera|  M| 53|2.62147382011095E9|    1|
| 150084|2015-04-28 21:13:25|2015-05-04 13:54:50|            44|ATGTXKYKUDUQN|   SEO| Safari|  M| 41|3.84054244391396E9|    0|
| 221365|2015-07-21 07:09:52|2015-09-09 18:40:53|            39|NAUITBZFJKHWW|   Ads| Safari|  M| 45|4.15583117

# **4. Checando o Schema e tratamento de dados:**

In [6]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- signup_time: timestamp (nullable = true)
 |-- purchase_time: timestamp (nullable = true)
 |-- purchase_value: integer (nullable = true)
 |-- device_id: string (nullable = true)
 |-- source: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- ip_address: double (nullable = true)
 |-- class: integer (nullable = true)



In [7]:
# Verifica valores nulos em cada coluna
nulos_por_coluna = df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns))

# Mostra o total de valores nulos em cada coluna
nulos_por_coluna.show()

#Caso haja algum valor nulo na tabela ele não seja significativo podemos utilizar o df.dropna()
#Caso seja muitos registros existem várias técnicas que podemos utilizar, como fillna(), coalesce() e impute()

#df = df.dropna()

+-------+-----------+-------------+--------------+---------+------+-------+---+---+----------+-----+
|user_id|signup_time|purchase_time|purchase_value|device_id|source|browser|sex|age|ip_address|class|
+-------+-----------+-------------+--------------+---------+------+-------+---+---+----------+-----+
|      0|          0|            0|             0|        0|     0|      0|  0|  0|         0|    0|
+-------+-----------+-------------+--------------+---------+------+-------+---+---+----------+-----+



In [8]:
#Conversão da coluna class para melhor entendimento do dataframe

#Converter os valores da coluna 'class' para string e atribuir à nova coluna 'fraude'
df = df.withColumn("fraude", when(col("class") == 1, "Sim").otherwise("Não"))

df = df.drop("class")
df.show()

+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+
|user_id|        signup_time|      purchase_time|purchase_value|    device_id|source|browser|sex|age|        ip_address|fraude|
+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+
|  22058|2015-02-24 22:55:49|2015-04-18 02:47:11|            34|QVPSPJUOCKZAR|   SEO| Chrome|  M| 39| 7.3275836879972E8|   Não|
| 333320|2015-06-07 20:39:50|2015-06-08 01:38:54|            16|EOGFQPIZPYXFZ|   Ads| Chrome|  F| 53|3.50311387865908E8|   Não|
|   1359|2015-01-01 18:52:44|2015-01-01 18:52:45|            15|YSSKYOSJHPPLJ|   SEO|  Opera|  M| 53|2.62147382011095E9|   Sim|
| 150084|2015-04-28 21:13:25|2015-05-04 13:54:50|            44|ATGTXKYKUDUQN|   SEO| Safari|  M| 41|3.84054244391396E9|   Não|
| 221365|2015-07-21 07:09:52|2015-09-09 18:40:53|            39|NAUITBZFJKHWW|   Ads| Safari|  M| 45|4.1

# **5. Análise inicial dos dados**


In [9]:
#Sumarizando os dados

df.summary().show()

+-------+------------------+-----------------+-------------+------+-------+------+-----------------+--------------------+------+
|summary|           user_id|   purchase_value|    device_id|source|browser|   sex|              age|          ip_address|fraude|
+-------+------------------+-----------------+-------------+------+-------+------+-----------------+--------------------+------+
|  count|            151112|           151112|       151112|151112| 151112|151112|           151112|              151112|151112|
|   mean|200171.04096961193|36.93537243898565|         NULL|  NULL|   NULL|  NULL|33.14070358409656|2.1521453309588385E9|  NULL|
| stddev|115369.28502413367|18.32276214866217|         NULL|  NULL|   NULL|  NULL|8.617733490961522|1.2484970301004207E9|  NULL|
|    min|                 2|                9|AAALBGNHHVMKG|   Ads| Chrome|     F|               18|    52093.4968949854|   Não|
|    25%|            100606|               22|         NULL|  NULL|   NULL|  NULL|               

In [10]:
#Definindo váriaveis globais para a fraude

df_fraude = df.filter(df["fraude"] == "Sim")
df_legitima = df.filter(df["fraude"] == "Não")

**É importante verificar que as compras fraudulentas correspondem a cerca de 9.36% das compras totais, isso causa um impacto significativo no financeiro das empresas de e-commerce, abaixo temos algumas informações e insights referentes ao valor das compras (purchase_value)**

É possível verificar que a média do valor de compras é parecido com as compras legitimas, isso pode ser uma forma de mascarar as compras fraudulentas em meio as legitimas, dessa forma o valor de compra não corresponde um varíavel tão significativa na identificação da fraude, sendo necessário a análise de outros fatores.

In [11]:
#Contando o número de compras fraudadas e legitimas(OBS: class = 1 é fraude, já = 0 é legítima).

df.groupBy("fraude").count().show()

#Porcentagem em relação ao total de compras fraudadas
porc_fraude = df_fraude.count() / df.count() * 100
print(porc_fraude)

#Porcentagem em relação ao total de compras legítimas.
porc_legitima = df_legitima.count() / df.count() * 100
print(porc_legitima)

+------+------+
|fraude| count|
+------+------+
|   Não|136961|
|   Sim| 14151|
+------+------+

9.364577267192546
90.63542273280746


In [52]:
#Valor de total de compra por compra fraudada e legitima.
df_vendas_tipo = df.groupBy("fraude").agg(sum("purchase_value").alias("Total Vendas"))

df_vendas_tipo.show()

+------+------------+
|fraude|Total Vendas|
+------+------------+
|   Não|     5057890|
|   Sim|      523488|
+------+------------+



In [13]:
#Resumo do valor de compras em relação ao indicador de fraude.

df_resumo = df.groupby("fraude").agg(
    sql.functions.mean("purchase_value").alias("valor_medio"),
    sql.functions.stddev("purchase_value").alias("desvio_padrao"),
    sql.functions.count("purchase_value").alias("contagem"),
)

df_resumo.show()

+------+------------------+------------------+--------+
|fraude|       valor_medio|     desvio_padrao|contagem|
+------+------------------+------------------+--------+
|   Não| 36.92941786347939| 18.31506387000993|  136961|
|   Sim|36.993004027983886|18.397653690084162|   14151|
+------+------------------+------------------+--------+



**A alta frequência de compras com apenas um segundo de diferença entre criação de conta e compra levanta a suspeita de um ataque "pump and dump". Nesse tipo de fraude, os fraudadores criam contas rapidamente, fazem compras fraudulentas e abandonam as contas, isso é ainda mais reforçado ao verificarmos que todas as compras tem um id de usuário (user_id) diferente**


Possíveis soluções seria análise da velocidade de digitação, além disso contas com velocidades anormais podem ser bloqueadas para análise manual ou através de uma IA, além disso verificação adicional de identidade, como autenticação multifator (MFA) ajudaria a reduzir esse tipo de método para compras fraudulentas.

In [14]:
#Verificando a diferença de dias e segundos para a criação da conta até a data da compra

df_datadif_dia = df_fraude.withColumn("dias_diferenca", datediff("purchase_time", "signup_time"))

df_datadif_segundos = df_fraude.withColumn("segundos_diferenca", unix_timestamp("purchase_time") - unix_timestamp("signup_time"))

In [15]:
df_datadif_dia.show()
df_datadif_segundos.show()

+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+--------------+
|user_id|        signup_time|      purchase_time|purchase_value|    device_id|source|browser|sex|age|        ip_address|fraude|dias_diferenca|
+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+--------------+
|   1359|2015-01-01 18:52:44|2015-01-01 18:52:45|            15|YSSKYOSJHPPLJ|   SEO|  Opera|  M| 53|2.62147382011095E9|   Sim|             0|
| 171711|2015-01-11 01:51:15|2015-01-11 01:51:16|            52|YPLQGKBAAULUV|Direct|     IE|  F| 35|1.12061933584996E9|   Sim|             0|
|  59780|2015-01-04 22:32:33|2015-01-16 01:13:43|            31|WUMOBGUCBOXPO|   SEO|FireFox|  M| 36| 1.7521668237958E9|   Sim|            12|
| 151705|2015-01-07 03:44:53|2015-01-07 03:44:54|            48|URHCRIXOMLJMH|   SEO| Chrome|  F| 27|2.83602522069469E9|   Sim|             0|

In [16]:
# Contando os valores distintos de user_id
df_usuarios_distintos = df.select("user_id").distinct().count()


# Imprimindo a quantidade de usuários distintos
print("Quantidade de usuários distintos:", df_usuarios_distintos)

Quantidade de usuários distintos: 151112


**Esse é um insight valioso e indica uma disparidade significativa entre o número de compras fraudulentas e legítimas em relação a um determinado dispositivo.**

Possíveis soluções seria bloquear dispositivos que realizam muitas compras em pouco tempo, além disso como podemos ver no total acumulado por dispositivo e na média móvel os dispositivos sempre fazem as compras com o mesmo valor, o que pode ser mais indício de um padrão no comportamento dos compradores fraudulentos.

In [17]:
#Dispositivos com mais compras fraudulentas

df_dispositivos_fraude = df_fraude.groupBy("device_id").agg(count("purchase_value").alias("Contagem"),round(avg("purchase_value"), 2).alias("Valor médio compras"))
df_dispositivos_fraude = df_dispositivos_fraude.orderBy("Contagem", ascending = False)
df_dispositivos_fraude.show()

+-------------+--------+-------------------+
|    device_id|Contagem|Valor médio compras|
+-------------+--------+-------------------+
|EQYVNEGOFLAWK|      19|              68.53|
|KIPFSCNUGOLDP|      19|               58.0|
|ITUMJCKWEYNDD|      19|               38.0|
|NGQCKIADMZORL|      19|               42.0|
|CQTUVBYIWWWBC|      19|               32.0|
|ZUSVMDEZRBDTX|      19|               47.0|
|SDJQRPKXQFBED|      18|               33.0|
|BWSMVSLCJXMCM|      18|               15.0|
|EGLGSEGYPMMAM|      18|               36.0|
|IGKYVZDBEGALB|      18|               26.0|
|CDFXVYHOIHPYP|      18|               44.0|
|OGBNHQHDZLGFZ|      17|               59.0|
|FFWAQIABHGYJC|      17|               11.0|
|XJWEQEWCBRAKD|      17|               24.0|
|GTIYVLCMAYBFA|      17|               14.0|
|QVMVTZOIJDKNR|      17|               59.0|
|XHZBVWFWHSGTQ|      17|               54.0|
|KPAAACGRQWYIK|      17|               81.0|
|RWZCXZTQUORQL|      17|               71.0|
|TAODVYWZT

In [18]:
#Dispositivos com mais compras legitimas.

df_dispositivos_legitima = df_legitima.groupBy("device_id").agg(count("purchase_value").alias("Contagem"),round(avg("purchase_value"), 2).alias("Valor médio compras"))
df_dispositivos_legitima = df_dispositivos_legitima.orderBy("Contagem", ascending = False)
df_dispositivos_legitima.show()

+-------------+--------+-------------------+
|    device_id|Contagem|Valor médio compras|
+-------------+--------+-------------------+
|FHKAGBGKXTAFT|       3|               36.0|
|MSXHVARAESYVD|       3|              37.67|
|VGJENUEZWZAOK|       3|               43.0|
|QKCFXAKWCADLJ|       3|              31.33|
|MJOUMICHIMYAP|       3|               21.0|
|CIWZEIWVKCTDZ|       3|               41.0|
|MDLRUHFIIHHPY|       3|               40.0|
|QFZJSWVRVKZAT|       3|              73.67|
|HVGLMDIRYIEVX|       3|              38.33|
|WKXDOKMKCXLNQ|       3|               31.0|
|MBXRPIGJLGMHU|       3|               53.0|
|BYCZKAGDTERRQ|       3|               29.0|
|HPJABLMUQASBK|       3|               19.0|
|TUTIBAJWVRPPI|       3|              60.33|
|ESTKJLLKCHBMW|       3|              51.33|
|FUJYPBUMWDXTO|       3|               38.0|
|LLVQVKNEEXFMS|       3|              35.33|
|NMWIJIQLRSPTN|       3|              53.33|
|CPSDECQLSMHGK|       3|              41.67|
|HCYSLYNRF

In [19]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("device_id").orderBy(desc("purchase_time"))

# Calculando o total acumulado o valor de compra
df_total_acumulado = df_fraude.withColumn(
    "Total Acumulado", sum("purchase_value").over(windowSpec)
)

# Calculando a média móvel do valor de compra
tam_janela = 3
df_media_movel = df_total_acumulado.withColumn(
    "Média Móvel",
    avg("purchase_value").over(windowSpec.rowsBetween(-tam_janela + 1, 0)),
)

# Exibindo os resultados
df_media_movel.show()

+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+---------------+-----------+
|user_id|        signup_time|      purchase_time|purchase_value|    device_id|source|browser|sex|age|        ip_address|fraude|Total Acumulado|Média Móvel|
+-------+-------------------+-------------------+--------------+-------------+------+-------+---+---+------------------+------+---------------+-----------+
|  77511|2015-01-01 04:25:31|2015-01-01 04:25:32|            57|AAAXXOZJRZRAO|   Ads|FireFox|  F| 36|1.37784923309361E9|   Sim|             57|       57.0|
| 190872|2015-01-01 04:25:30|2015-01-01 04:25:31|            57|AAAXXOZJRZRAO|   Ads|FireFox|  F| 36|1.37784923309361E9|   Sim|            114|       57.0|
| 314594|2015-01-01 04:25:29|2015-01-01 04:25:30|            57|AAAXXOZJRZRAO|   Ads|FireFox|  F| 36|1.37784923309361E9|   Sim|            171|       57.0|
| 254993|2015-01-01 04:25:28|2015-01-01 04:25:29|            57|

# **6. Gravando o resultado em Parquet**

In [37]:
#Gravando o resultado e apresentando mensagem caso dê erro.
path = "/content/dataframeFinal"
try:
    df_media_movel.write.option("mode", "overwrite").parquet(path)
    print("Arquivo salvo com sucesso!")
except Exception as e:
    print("Erro ao salvar o arquivo:", e)

display(path)

Arquivo salvo com sucesso!


# **7. Conclusão**

Esta análise exploratória de dados fornece insights valiosos sobre a fraude em compras online, utilizando informações sobre valor da compra, frequência de compras, dispositivos utilizados e comportamento do usuário.

**1. 9.36% das compras são fraudulentas: Essa porcentagem, embora pareça pequena, representa um impacto significativo nas finanças das empresas de e-commerce.**

**2. Valor médio similar entre compras legítimas e fraudulentas: O valor da compra por si só não se mostra um indicador confiável para identificar fraudes, pois a média é similar em ambos os tipos de compra. Os fraudadores mascaram suas ações imitando o comportamento dos compradores legítimos.**

**3. Ataque "pump and dump": A alta frequência de compras com apenas um segundo de diferença entre criação de conta e compra levanta a suspeita desse tipo de ataque. Os fraudadores criam contas rapidamente, realizam compras fraudulentas e as abandonam, apesar de manterem o mesmo dispositivo, o que pode ser uma ótima métrica para mitigar os prejuízos.**

**4. A média móvel constante pode indicar que um fraudador está usando um script ou bot para realizar compras automatizadas.**

Algumas soluções apresentadas incluem: **Análise da velocidade na criação de conta e compra ajudando na identificação de bots e scripts, bloqueio de disposítivos que apresentam multiplas compras com o mesmo valor em baixíssimo período e verificação de multifator (MFA) para dificultar na criação de multiplas contas.**

Esta análise fornece uma visão geral da fraude em compras online e serve como base para investigações mais aprofundadas.